In [ ]:
import torch
#print(torch.cuda.is_available())
#print(torch.cuda.get_device_name(0))


In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch


In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv", sep=",", engine="python", on_bad_lines="skip")
val_df = pd.read_csv("validation.csv", sep=",", engine="python", on_bad_lines="skip")
test_df = pd.read_csv("test.csv", sep=",", engine="python", on_bad_lines="skip")



In [ ]:
#print(train_df.columns)
#print(train_df.head(3))


In [ ]:
print(train_df.columns)
print(train_df.head())


In [ ]:
train_df = train_df.rename(columns={"article": "text", "highlights": "summary"})
val_df   = val_df.rename(columns={"article": "text", "highlights": "summary"})
test_df  = test_df.rename(columns={"article": "text", "highlights": "summary"})


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

train_sample = train_df.sample(n=200, random_state=42)
val_sample = val_df.sample(n=100, random_state=42)
test_sample = test_df.sample(n=100, random_state=42)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")


In [ ]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs




In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
print(train_sample.columns)


In [ ]:
#max_input_length = 256
#max_target_length = 64

#def preprocess_function(examples):
 #   inputs = examples["text"]
  #  targets = examples["summary"]

   # model_inputs = tokenizer(
    #    inputs,
     #   max_length=512,
      #  padding="max_length",
       # truncation=True
    #)

    #labels = tokenizer(
     #   targets,
      #  max_length=128,
       # padding="max_length",
        #truncation=True
   # )

   # model_inputs["labels"] = labels["input_ids"]
    #return model_inputs


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_sample, preserve_index=False)
val_dataset = Dataset.from_pandas(val_sample, preserve_index=False)
test_dataset = Dataset.from_pandas(test_sample, preserve_index=False)


In [ ]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        targets,
        max_length=128,
        padding="max_length",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
print(train_sample.columns)
print(train_sample.head())


In [ ]:
print(train_dataset.features)


In [ ]:
#!pip install transformers==4.40.0 --upgrade


In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure,
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    # ... other arguments ...
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
#trainer = Trainer(
 #   model=model,
  #  args=training_args,
   # train_dataset=train_dataset,
    #eval_dataset=val_dataset,
    #tokenizer=tokenizer,
    #compute_metrics=compute_metrics
#)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

train_sample = train_df.sample(n=200, random_state=42)
val_sample = val_df.sample(n=100, random_state=42)
test_sample = test_df.sample(n=100, random_state=42)


train_dataset = Dataset.from_pandas(train_sample)
val_dataset = Dataset.from_pandas(val_sample)
test_dataset = Dataset.from_pandas(test_sample)


train_dataset = train_dataset.map(preprocess_function, batched=True, batch_size=4)
val_dataset = val_dataset.map(preprocess_function, batched=True, batch_size=4)
test_dataset = test_dataset.map(preprocess_function, batched=True, batch_size=4)

training_args =  TrainingArguments (
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
     report_to="none",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #fp16=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<ipython-input-24-96b3aaccc5ae>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
from datasets import load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

rouge = load_metric("rouge")

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("./results").to("cuda" if torch.cuda.is_available() else "cpu")

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(model.device)
    summary_ids = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

test_data = [
    {"input": "Modern technology is one of the most important factors...",
     "target": "excessive use of technology can lead to decreased social interaction..."},

]

def evaluate_rouge(dataset):
    predictions = []
    references = []
    for sample in dataset:
        pred = generate_summary(sample["input"])
        predictions.append(pred)
        references.append(sample["target"])
    results = rouge.compute(predictions=predictions, references=references)
    return results

print("\n--- Test ROUGE ---")
test_scores = evaluate_rouge(test_data)
for key in test_scores:
    score = test_scores[key].mid
    print(f"{key.upper()} - Precision: {score.precision:.2f}, Recall: {score.recall:.2f}, F1: {score.fmeasure:.2f}")


In [ ]:
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

log_history = trainer.state.log_history
df_log = pd.DataFrame(log_history)

plt.figure(figsize=(10, 6))
if "loss" in df_log.columns:
    plt.plot(df_log[df_log["loss"].notna()]["step"], df_log[df_log["loss"].notna()]["loss"], label="Training Loss")
if "eval_loss" in df_log.columns:
    plt.plot(df_log[df_log["eval_loss"].notna()]["step"], df_log[df_log["eval_loss"].notna()]["eval_loss"], label="Validation Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Loss During Training")
plt.legend()
plt.grid(True)
plt.show()

if "eval_rouge1" in df_log.columns:
    plt.figure(figsize=(10, 6))
    plt.plot(df_log["step"], df_log["eval_rouge1"], label="ROUGE-1")
    plt.plot(df_log["step"], df_log["eval_rouge2"], label="ROUGE-2")
    plt.plot(df_log["step"], df_log["eval_rougeL"], label="ROUGE-L")
    plt.xlabel("Step")
    plt.ylabel("Score")
    plt.title("ROUGE Scores During Evaluation")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("ROUGE scores are not available in the log history.")


In [ ]:
#print(df_log.columns)
#print(df_log.head())


In [ ]:
eval_results = trainer.evaluate(test_dataset)
print("Evaluation Results:", eval_results)


In [ ]:
from evaluate import load

rouge = load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure,
    }



In [ ]:
!pip install rouge_score


In [ ]:
!pip install evaluate

from evaluate import load

rouge = load("rouge")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

logs = pd.DataFrame(trainer.state.log_history)
logs.plot(x="epoch", y=["loss", "eval_loss"])
plt.title("Training vs Validation Loss")
plt.show()


In [ ]:
def summarize_text(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    output_ids = model.generate(inputs["input_ids"].to(model.device), max_length=150, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

input_text = input("اكتب النص اللي عايز تلخصه:\n")


predicted_summary = summarize_text(input_text)

real_summary = ""

print("\nالنص الأصلي:\n", input_text)
print("\nالتلخيص المتوقع:\n", predicted_summary)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from evaluate import load
import pandas as pd
import torch

model_path = "./results"
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

def summarize_text(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    input_ids = inputs["input_ids"].to(model.device)

    output_ids = model.generate(
        input_ids,
        max_length=150,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("validation.csv")

assert len(df_test) == len(df_val)
rouge = load("rouge")

for i in range(len(df_test)):
    input_text = df_test.iloc[i]["article"]
    reference_summary = df_val.iloc[i]["highlights"]
    predicted_summary = summarize_text(input_text)

    rouge.add(prediction=predicted_summary, reference=reference_summary)

results = rouge.compute()

print("\n\n===== تقييم النموذج على مقارنة test و validation =====")
for key in results:
    print(f"{key}: {results[key].mid.fmeasure:.4f}")
